Initialize the libararies

In [8]:
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.components.optimizers import L_BFGS_B
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.tools.visualization import plot_histogram
from qiskit import IBMQ, execute
from qiskit.circuit.library import EfficientSU2
from qiskit.providers.aer.noise import NoiseModel
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import numpy as np
from qiskit.aqua.components.optimizers import SPSA

Use PySCF, a classical computational chemistry software package, to compute the one-body and two-body integrals in
molecular-orbital basis, necessary to form the Fermionic operator

In [9]:
driver = PySCFDriver(atom='C .0 .0 .0;O .0 .0 .93', unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()
num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2

The Solver
Then we need to define a solver. The solver is the algorithm through which the ground state is computed.
Let's first start with a purely classical example: the NumPy minimum eigensolver. This algorithm exactly diagonalizes the Hamiltonian. Although it scales badely, it can be used on small systems to check the results of the quantum algorithms.

In [ ]:
# Build the qubit operator, which is the input to the VQE algorithm in Aqua
ferm_op = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
map_type = 'JORDAN_WIGNER'
qubit_op = ferm_op.mapping(map_type)
qubit_op = Z2Symmetries.two_qubit_reduction(qubit_op, num_particles)
num_qubits = qubit_op.num_qubits
print(num_qubits)

Let's initialize a VQE solver.

In [4]:
# setup a classical optimizer for VQE
optimizer = SPSA(maxiter=100)
#optimizer = L_BFGS_B()

To define the VQE solver one needs two essential elements:

A variational form: here we use the Unitary Coupled Cluster (UCC) ansatz (see for instance [Physical Review A 98.2 (2018): 022322]). Since it is a chemistry standard, a factory is already available allowing a fast initialization of a VQE with UCC. The default is to use all single and double excitations. However, the excitation type (S, D, SD) as well as other parameters can be selected.
An initial state: the initial state of the qubits. In the factory used above, the qubits are initialized in the Hartree-Fock (see the electronic structure tutorial) initial state (the qubits corresponding to occupied MOs are $|1\rangle$ and those corresponding to virtual MOs are $|0\rangle$.
The backend: this is the quantum machine on which the right part of the figure above will be performed. Here we ask for the perfect quantum emulator (statevector_simulator).
One could also use any available variational form / initial state or even define one's own. For instance,

In [5]:
# setup the initial state for the variational form

init_state = HartreeFock(num_spin_orbitals, num_particles)

The calculation and results
We are now ready to run the calculation.

In [6]:
# setup the variational form for VQE
from qiskit.circuit.library import TwoLocal
var_form = EfficientSU2(qubit_op.num_qubits, entanglement="linear")

# setup and run VQE
from qiskit.aqua.algorithms import VQE
algorithm = VQE(qubit_op, var_form, optimizer)

# set the backend for the quantum computation
from qiskit import Aer
#backend = Aer.get_backend('statevector_simulator')
backend = Aer.get_backend('qasm_simulator')
result = algorithm.run(backend)
vqe_result=np.real(result['eigenvalue']+molecule.nuclear_repulsion_energy)
print("VQE REsult:",vqe_result)


VQE REsult: -0.7395626092278245


Running on the IBMQ

In [54]:
IBMQ.delete_account()

In [55]:
IBMQ.active_account()

{'token': 'beb2ac53efd858042d3dbdc7caa009b058b83b6e65a818fd9923371ade1a0e81e92cdd3d7541ac1735b7fb1477ec2343a1e7fdba5cc8051fcae7b116d4dc31a9',
 'url': 'https://auth.quantum-computing.ibm.com/api'}

In [56]:
IBMQ.save_account('beb2ac53efd858042d3dbdc7caa009b058b83b6e65a818fd9923371ade1a0e81e92cdd3d7541ac1735b7fb1477ec2343a1e7fdba5cc8051fcae7b116d4dc31a9', overwrite=True)

In [113]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = Aer.get_backend("qasm_simulator") 
device = provider.get_backend("ibmq_vigo")  # (other backends are available as well, if you want to go looking)
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device.properties())
quantum_instance = QuantumInstance(backend = backend,
                                  shots = 8192,
                                  noise_model = noise_model,
                                  coupling_map=coupling_map,
                                  measurement_error_mitigation_cls=CompleteMeasFitter,
                                   cals_matrix_refresh_period = 30)

ibmqfactory.load_account:WARNING:2020-11-10 13:09:56,823: Credentials are already in use. The existing account in the session will be replaced.


In [7]:
exact_solution = NumPyEigensolver(qubit_op).run()
print("Exact Result:",np.real(exact_solution.eigenvalues)+molecule.nuclear_repulsion_energy)
var_form = EfficientSU2(qubit_op.num_qubits, entanglement="linear")
vqe = VQE(qubit_op,var_form,optimizer=optimizer)
ret = vqe.run(quantum_instance)
vqe_result=np.real(ret['eigenvalue']+molecule.nuclear_repulsion_energy)
print("VQE REsult:",vqe_result)


Exact Result: [-0.74361971]


NameError: name 'quantum_instance' is not defined

Using a filter function
Sometimes the true ground state of the Hamiltonian is not of interest because it lies in a different symmetry sector of the Hilbert space. In this case the NumPy eigensolver can take a filter function to return only the eigenstates with for example the correct number of particles. This is of particular importance in the case of vibronic structure calculations where the true ground state of the Hamiltonian is the vacuum state. A default filter function to check the number of particles is implemented in the different transformations and can be used as

In [ ]:
from qiskit.chemistry.drivers import GaussianForcesDriver
from qiskit.chemistry.algorithms.ground_state_solvers import NumPyMinimumEigensolverFactory
from qiskit.chemistry.transformations import (BosonicTransformation,
                                              BosonicTransformationType,
                                              BosonicQubitMappingType)

driver = GaussianForcesDriver(logfile='aux_files/CO2_freq_B3LYP_ccpVDZ.log')
bosonic_transformation = BosonicTransformation(qubit_mapping=BosonicQubitMappingType.DIRECT,
                                               transformation_type=BosonicTransformationType.HARMONIC,
                                               basis_size=2,
                                               truncation=2)

solver_without_filter = NumPyMinimumEigensolverFactory(use_default_filter_criterion=False)
solver_with_filter = NumPyMinimumEigensolverFactory(use_default_filter_criterion=True)

gsc_wo = GroundStateEigensolver(bosonic_transformation, solver_without_filter)
result_wo = gsc_wo.solve(driver)

gsc_w = GroundStateEigensolver(bosonic_transformation, solver_with_filter)
result_w = gsc_w.solve(driver)

print(result_wo)
print('\n\n')
print(result_w)